In [ ]:
import tensorflow as tf
import numpy as np

class Generator(tf.keras.Model):
    def __init__(self):
        super(Generator, self).__init__()

        self.encoder = tf.keras.Sequential([
            Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 1)),
            MaxPooling2D((2, 2)),
            Conv2D(64, (3, 3), activation='relu'),
            MaxPooling2D((2, 2)),
            Conv2D(128, (3, 3), activation='relu'),
            MaxPooling2D((2, 2)),
        ])

        self.decoder = tf.keras.Sequential([
            UpSampling2D((2, 2)),
            Conv2D(128, (3, 3), activation='relu'),
            UpSampling2D((2, 2)),
            Conv2D(64, (3, 3), activation='relu'),
            UpSampling2D((2, 2)),
            Conv2D(32, (3, 3), activation='relu'),
            Conv2D(1, (3, 3), activation='sigmoid')
        ])

    def call(self, inputs):
        x = self.encoder(inputs)
        x = self.decoder(x)
        return x

class Discriminator(tf.keras.Model):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.encoder = tf.keras.Sequential([
            Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 1)),
            MaxPooling2D((2, 2)),
            Conv2D(64, (3, 3), activation='relu'),
            MaxPooling2D((2, 2)),
            Conv2D(128, (3, 3), activation='relu'),
            MaxPooling2D((2, 2)),
        ])

        self.flatten = tf.keras.layers.Flatten()
        self.dense1 = tf.keras.layers.Dense(128, activation='relu')
        self.dense2 = tf.keras.layers.Dense(1, activation='sigmoid')

    def call(self, inputs):
        x = self.encoder(inputs)
        x = self.flatten(x)
        x = self.dense1(x)
        x = self.dense2(x)
        return x

# Load the preprocessed data
(X_train, y_train), (X_test, y_test) = np.load('preprocessed_nuclei_dataset.npy')

# Define the CGAN model
generator = Generator()
discriminator = Discriminator()

# Compile the CGAN model
generator.compile(loss='binary_crossentropy', optimizer='adam')
discriminator.compile(loss='binary_crossentropy', optimizer='adam')

# Train the CGAN model
for epoch in range(320):
    # Sample a batch of real nuclei images from the training set
    real_images = X_train[np.random.randint(0, X_train.shape[0], 32)]

    # Generate synthetic nuclei images using the generator network
    fake_images = generator.predict(real_images)

    # Train the discriminator network
    discriminator.trainable = True
    d_loss_real = discriminator.train_on_batch(real_images, np.ones((32, 1)))
    d_loss_fake = discriminator.train_on_batch(fake_images, np.zeros((32, 1)))
    d_loss = (d_loss_real + d_loss_fake) / 2

    # Train the generator network
    discriminator.trainable = False
    g_loss = generator.train_on_batch(real_images, np.ones((32, 1)))

    # Print the training loss
    print(f'Epoch: {epoch + 1}, d_loss: {d_loss}, g_loss: {g_loss}')

# Save the CGAN model
generator.save('cgan_model.h5')

# Test the CGAN model
dice, sensitivity, specificity, f1_score = test_cgan(generator,


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Concatenate, Activation

class Generator(tf.keras.Model):
    def __init__(self):
        super(Generator, self).__init__()

        self.encoder = tf.keras.Sequential([
            Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 1)),
            MaxPooling2D((2, 2)),
            Conv2D(64, (3, 3), activation='relu'),
            MaxPooling2D((2, 2)),
            Conv2D(128, (3, 3), activation='relu'),
            MaxPooling2D((2, 2)),
        ])

        self.decoder = tf.keras.Sequential([
            UpSampling2D((2, 2)),
            Conv2D(128, (3, 3), activation='relu'),
            UpSampling2D((2, 2)),
            Conv2D(64, (3, 3), activation='relu'),
            UpSampling2D((2, 2)),
            Conv2D(32, (3, 3), activation='relu'),
            Conv2D(1, (3, 3), activation='sigmoid')
        ])

    def call(self, inputs):
        x = self.encoder(inputs)
        x = self.decoder(x)
        return x

class Discriminator(tf.keras.Model):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.encoder = tf.keras.Sequential([
            Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 1)),
            MaxPooling2D((2, 2)),
            Conv2D(64, (3, 3), activation='relu'),
            MaxPooling2D((2, 2)),
            Conv2D(128, (3, 3), activation='relu'),
            MaxPooling2D((2, 2)),
        ])

        self.flatten = tf.keras.layers.Flatten()
        self.dense1 = tf.keras.layers.Dense(128, activation='relu')
        self.dense2 = tf.keras.layers.Dense(1, activation='sigmoid')

    def call(self, inputs):
        x = self.encoder(inputs)
        x = self.flatten(x)
        x = self.dense1(x)
        x = self.dense2(x)
        return x


In [ ]:
import tensorflow as tf

def train_cgan(generator, discriminator, X_train):
    # Adam optimizer with learning rate = 0.0002
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002)

    # Train the discriminator network
    discriminator.compile(loss='binary_crossentropy', optimizer=optimizer)

    # Train the generator network
    generator.compile(loss='binary_crossentropy', optimizer=optimizer)

    # Number of epochs to train the model
    epochs = 320

    # Start training loop


In [ ]:
import tensorflow as tf
import numpy as np

def test_cgan(generator, X_test):
    # Generate synthetic nuclei images using the generator network
    y_pred = generator.predict(X_test)

    # Compute the dice coefficient
    y_pred_binary = np.round(y_pred)
    dice = np.mean(2 * np.sum(y_pred_binary * X_test, axis=(1, 2)) / (np.sum(y_pred_binary, axis=(1, 2)) + np.sum(X_test, axis=(1, 2))))

    # Compute the sensitivity and specificity
    sensitivity = np.mean(np.sum(y_pred_binary * X_test, axis=(1, 2)) / np.sum(X_test, axis=(1, 2)))
    specificity = np.mean(np.sum((1 - y_pred_binary) * (1 - X_test), axis=(1, 2)) / np.sum(1 - X_test, axis=(1, 2)))

    # Compute the F1 score
    f1_score = 2 * sensitivity * precision / (sensitivity + precision)

    return dice, sensitivity, specificity, f1_score

# Load the CGAN model
generator = tf.keras.models.load_model('cgan_model.h5')

# Load the test set
X_test, y_test = np.load('test_nuclei_dataset.npy')

# Test the CGAN model
dice, sensitivity, specificity, f1_score = test_cgan(generator, X_test)

# Print the results
print('Dice coefficient:', dice)
print('Sensitivity:', sensitivity)
print('Specificity:', specificity)
print('F1 score:', f1_score)
